In [1]:
from ete3 import Tree

In [82]:
t = Tree('LOV_tree', format=0)

In [75]:
t = Tree('(((H:0,K:0):0.5,(F:1,I:1):0):1,E:2);', format=5)

In [83]:
s = []
for n in t.traverse():
    if not n.is_leaf():
        if all(child.is_leaf() and child.dist == 0 for child in n.children):
            s.append(n.children[0])
        else:
            for c in n.children:
                if c.is_leaf():
                    s.append(c)

In [85]:
t.prune(s, preserve_branch_length=True)

In [81]:
print(t.write())

(((F:1,I:1)1:0,H:0.5)1:1,E:2);


In [54]:
not t.is_leaf()

True

In [84]:
len(s)

9321

In [70]:
len(t.get_leaves())

12646

In [86]:
with open('LOV_tree_1', 'w') as f:
    f.write(t.write())

In [91]:
for i in t.iter_search_nodes(support=0):
    i.delete()

In [92]:
with open('LOV_tree_2', 'w') as f:
    f.write(t.write())

In [93]:
t.get_newick()

AttributeError: 'TreeNode' object has no attribute 'get_newick'

In [94]:
t.write(format=0, outfile='LOV_tree_2')

In [99]:
with open('LOV_tree_2', 'r') as f:
    line = f.readline()

In [105]:
l = line.split(':')
l = [i.split(',') for i in l]

In [106]:
for i in l:
    if len(i) == 2:
        i[0]

[['((A0A126WYJ30'],
 ['0.0106367', '((A0A126WYF21'],
 ['5e-09', '(A0A126WVX70'],
 ['0.0106113', '(A0A126X3S10'],
 ['5e-09', 'A0A126X3S50'],
 ['5e-09)0.935'],
 ['0.0218831', '(A0A126X0191'],
 ['5e-09', 'A0A126WY961'],
 ['0.0107941)0.903'],
 ['0.0107596)0.931'],
 ['0.0106266)0.881'],
 ['0.0105953', '((A0A126WXK71'],
 ['5e-09', '(A0A126X2Q40'],
 ['0.0328199', 'A0A126X1B81'],
 ['5e-09)0.414'],
 ['0.0105877)0.858'],
 ['0.0105781', '(((A0A140F7K51'],
 ['0.0216419', 'A0A126X1S51'],
 ['0.010897', '(A0A2P6PTQ01'],
 ['5e-09', 'A0A126WZL71'],
 ['0.0105094', 'J7MDN81'],
 ['0.0109152)0.922'],
 ['0.0106223', 'A0A126X4C61'],
 ['5e-09)0.91'],
 ['0.010628', '(A0A126X0530'],
 ['0.0545229', '(A0A0D2SYD61'],
 ['5e-09', 'A0A0B0MLL51'],
 ['5e-09)0.773'],
 ['0.0154657)0.812'],
 ['0.0171299', 'A0A140F7P40'],
 ['0.0326363', 'A0A126WWX41'],
 ['5e-09', '(A0A126WY111'],
 ['5e-09', 'A0A2H5P1F91'],
 ['0.0105308)0.852'],
 ['0.010629)0.93'],
 ['0.0105917', '(((A0A126X2401'],
 ['5e-09', 'A0A126X1L81'],
 ['0.0212919)0.

In [107]:
float(5e-09)

5e-09